In [ ]:
from ipywidgets import Button, HBox, HTML, Accordion, IntSlider, Text, Textarea, VBox
from Deque import Deque
from ipyevents import Event
from traitlets import Int, observe, link, Any

In [ ]:
class ScrollableButton(Button):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def update(self, data):
        self.description = data

In [ ]:
a = Deque(ScrollableButton, ["HELLO","GOODBYE","my","name","is"]*3)

In [ ]:
a

In [ ]:
b = Accordion([a])

In [ ]:
b

In [ ]:
class NodeInner(Accordion):
    def __init__(self, name, children=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.name = name
        self.children = children if children else []
        for i,c in enumerate(self.children):
            self.set_title(i,c.sections.name)
        
class Node(VBox):
    selected_index = Any().tag(sync=True)
    
    def __init__(self, *args,**kwargs):
        super().__init__()
        
        self.sections = NodeInner(*args,**kwargs)
        self.add_section_btn = Button(icon = "plus")
        self.add_section_btn.on_click(self.add_section)
        
        link((self.sections, "selected_index"), (self, "selected_index"))
        
        if self.selected_index is not None:
            self.children = [self.sections]
        else:
            self.children = [self.sections, self.add_section_btn]
        
    def add_section(self, _):
        self.sections.children = list(self.sections.children) + [Node("Heading?")]
        N = len(self.sections.children) - 1
        self.sections.set_title(N, "<Use tool to select title>")
        self.sections.selected_index = N

    @observe("selected_index")
    def set_btn_visibility(self, change):
        "Hide button when a subsection is selected"
        
        if change["new"] is None:
            self.children = [self.sections, self.add_section_btn]
        else:
            self.children = [self.sections]
            
    def path_to_selected(self):
        "Returns a list of indexes showing how to traverse the tree to get to the selected node"
        
        if self.selected_index is None:
            return []
        return [self.selected_index] + self.sections.children[self.selected_index].path_to_selected()

In [ ]:
n = Node(
    "root",
    [
        Node("section 1",
            [
                Node("section 1"), 
                Node("section 2",
            [
                Node("section 1"), 
                Node("section 2"),
                Node("section 3"), 
                Node("section 4")
            ]),
                Node("section 3"), 
                Node("section 4")
            ]
        ), 
        Node("section 2"),
        Node("section 3"), 
        Node("section 4")
    ],
)

In [ ]:
n

In [ ]:
n.path_to_selected()

In [ ]:
def Tree(data):
    if data:
        return [Node(d["label"], Tree(d["children"])) for d in data]



data =[
    {
        "label":"root", 
        "children":[
            {"label": "1", "children": []},
            {"label": "2", "children": []},
            {"label": "3", "children": []},
            {"label": "4", "children": []},
        ]
    }
]

In [ ]:
Tree(data)[0]